In [1]:
import re 
import copy
import itertools


#print function
def p(t):
    for l in t:
        print("".join(l))
    print("---------------------------------")
    
#pass input file to tile dictionary
tiles = {}
t = None
for l in re.findall("[0-9]+|[.#]+",open("i20.txt").read()):
    if l.isnumeric():
        t = int(l) 
        tiles[t] = []
    else:
        tiles[t].append(list(l))
        
print("there are %d tiles"%(len(tiles.keys())))
gl = int(len(tiles.keys())**0.5)
print("that is a sidelength of %d"%(gl))
print("each tile is %d by %d"%(len(list(tiles.values())[0][0]),len(list(tiles.values())[0])))



#manipulation
def hflip(src,s,x,y):
    return src[y][s-x-1]

def cw(src,s,x,y):
    return src[s-x-1][y]

def op(t, F):
    s = len(t)
    n = copy.deepcopy(t)
    for y in range(s):
        for x in range(s):
            n[y][x] = F(t,s,x,y)
    return n

operations = " rrrhrrr"

def manip(t, ops):
    for o in ops:
        if o == 'h':
            t = op(t, hflip)
        elif o == 'r':
            t = op(t, cw)
        elif o == ' ':
            pass
        else:
            print("unknown operation")
    return t





there are 144 tiles
that is a sidelength of 12
each tile is 10 by 10


In [2]:
#convert edge to number for easy comparison
def b2i(s):
    return int(s.replace(".", "0").replace("#","1"),2)

#top row
def u(z):
    return b2i("".join(z[0]))
#bottom row
def d(z):
    return b2i("".join(z[-1]))
#left column
def l(z):
    return b2i("".join([z[x][0] for x in range(len(z))]))
#right column
def r(z):
    return b2i("".join([z[x][-1] for x in range(len(z))]))


In [3]:

alledges = []

#pickup all edges in a dict with piece number as key
edges = {}
for tk in tiles.keys():
    tile = tiles[tk]
    edges[tk] = []
    for o in operations:
        tile = manip(tile, o) 
        edges[tk].append(u(tile))
        edges[tk].append(d(tile))
        edges[tk].append(l(tile))
        edges[tk].append(r(tile))
        alledges += edges[tk] 

print(len(alledges), len(list(set(alledges))))        

#Walk through all pieces and find other pieces that share same edge values
neighbours= {}
for e0 in edges.keys():
    neighbours[e0] = []
    for e1 in edges.keys():
        if e0 != e1:
            if set(edges[e0]).intersection(set(edges[e1])):
                neighbours[e0].append(e1)

#Check if any only has two
twos = []
threes = []
fours = []
others = []
for n in neighbours.keys():
    if len(neighbours[n]) == 2:
        twos.append(n)
    elif len(neighbours[n]) == 3:
        threes.append(n)
    elif len(neighbours[n]) == 4:
        fours.append(n)
    else:
        others.append(n)

print("twos", len(twos))
print("threes", len(threes))
print("fours", len(fours))
print("others", len(others))
        
#twos must be corners so we can answer part1
acc = 1
for tw in twos:
    acc*=tw
print("part1 :", acc, 27798062994017)


20736 624
twos 4
threes 40
fours 100
others 0
part1 : 27798062994017 27798062994017


In [4]:
#it seems odd that there is no cases of tiles with more than 4 neighbour candidates. 
#Its seems all connections are unique.

#pick one piece and search for other pieces and rotations that would match.
#also keep track of position

remaining = list(tiles.keys())

seed = remaining[0]
remaining = remaining[1:]

placed = {seed : [0, 0]}
open_u = { u(tiles[seed]) : [ 0,-1] }
open_d = { d(tiles[seed]) : [ 0, 1] }
open_l = { l(tiles[seed]) : [-1, 0] }
open_r = { r(tiles[seed]) : [ 1, 0] }

while len(remaining):
    rr = remaining[0]
    remaining = remaining[1:] 
    
    match=0
    for o in operations:
        tiles[rr] = manip(tiles[rr], o)
        vu = u(tiles[rr])
        vd = d(tiles[rr])
        vl = l(tiles[rr])
        vr = r(tiles[rr])
        
        if vd in open_u.keys():
            x,y = open_u[vd]
            del open_u[vd]
            
            if vr in open_l.keys():
                del open_l[vr]
            
            if vl in open_r.keys():
                del open_r[vl]
            
            open_u[vu] = [x  ,y-1]
            open_l[vl] = [x-1,y  ]
            open_r[vr] = [x+1,y  ]
            
            placed[rr] = [x, y]
            match=1
            break
        elif vu in open_d.keys():
            x,y = open_d[vu]
            del open_d[vu]
            
            if vr in open_l.keys():
                del open_l[vr]
            
            if vl in open_r.keys():
                del open_r[vl]
            
            open_d[vd] = [x  ,y+1]
            open_l[vl] = [x-1,y  ]
            open_r[vr] = [x+1,y  ]
            
            placed[rr] = [x, y]
            match=1
            break
        elif vr in open_l.keys():
            x,y = open_l[vr]
            del open_l[vr]
            
            if vd in open_u.keys():
                del open_u[vd]
            
            if vu in open_d.keys():
                del open_d[vu]
            
            open_l[vl] = [x-1,y  ]
            open_u[vu] = [x  ,y-1]
            open_d[vd] = [x  ,y+1]
            
            placed[rr] = [x, y]
            match=1
            break
        elif vl in open_r.keys():
            x,y = open_r[vl]
            del open_r[vl]
            
            if vd in open_u.keys():
                del open_u[vd]
            
            if vu in open_d.keys():
                del open_d[vu]
            
            open_r[vr] = [x+1,y  ]
            open_u[vu] = [x  ,y-1]
            open_d[vd] = [x  ,y+1]
            
            placed[rr] = [x, y]
            match=1
            break
        
        
    if match==0:
        remaining.append(rr)
    #print(rr,len(placed.keys()), len(remaining), len(open_u.keys()), len(open_d.keys()), len(open_l.keys()), len(open_r.keys()), end='\r')

#print("\nall rotated")
            
min_x = 0
min_y = 0
for k in placed.keys():
    x,y = placed[k]
    min_x = min(min_x, x)
    min_y = min(min_y, y)

#print(min_x, min_y)

p1 = 1

M = [[None for x in range(gl)] for y in range(gl)]
for k in placed.keys():
    x,y = placed[k]
    x += -min_x
    y += -min_y
    
    M[y][x] = tiles[k]
    
    #print(x,y)
    
    if x == 0 and y == 0:
        p1 *= k
    if x == gl-1 and y == 0:
        p1 *= k
    if x == 0 and y == gl-1:
        p1 *= k
    if x == gl-1 and y == gl-1:
        p1 *= k

print("Part1 (again):", p1)

#print(M)

#make map without tile edges
sl = gl * 8
C = [['?' for x in range(sl)] for y in range(sl)]
#fill in core values from tiles
for y in range(sl):
    for x in range(sl):
        mx = x // 8
        xx = x % 8+1
        my = y // 8
        yy = y % 8+1
        C[y][x] = M[my][mx][yy][xx]

#map seamonster
sm = []
sm.append(list("                  # "))
sm.append(list("#    ##    ##    ###"))
sm.append(list(" #  #  #  #  #  #   "))
#p(sm)    

SM = []
ox,oy = 0,1 
for y in range(len(sm)):
    for x in range(len(sm[0])):
        if sm[y][x] == '#':
            SM.append([x-ox,y-oy])
#print(SM)
    


        
for o in operations:
    #print("manip")
    got_seamonster = 0
    C = manip(C,o)
    for y in range(len(C)):
        for x in range(len(C[0])):
            if C[y][x] == '#':
                found = True
                for z in range(len(SM)):
                    sx,sy = SM[z]
                    xx = x + sx
                    yy = y + sy
                    if xx > 0 and xx < len(C[0]) and yy > 0 and yy < len(C):
                        if C[yy][xx] != '#':
                            found = False
                            break
                    else:
                        found = False
                if found:
                    #print("got seamonster")
                    got_seamonster += 1
                    
                    #make sure we dont doublecount seamonsters
                    for z in range(len(SM)):
                        sx,sy = SM[z]
                        xx = x + sx
                        yy = y + sy
                        C[yy][xx] = 'O'
                    
    
    if got_seamonster:
        acc = 0
        for y in range(len(C)):
            for x in range(len(C[0])):
                if C[y][x] == '#':
                    acc += 1
        print("monsters:", got_seamonster)
        print("part2: ", acc, 0)
        break
    
#p(C)

#3263, too high
#2456, too high

Part1 (again): 27798062994017
monsters: 38
part2:  2366 0
